In [1]:
from pandas import read_csv
from matplotlib import pyplot
import seaborn as sn
import matplotlib.pyplot as plt

import numpy as np
from sklearn import svm

In [2]:
# MMD implementation
# https://github.com/jindongwang/transferlearning/blob/master/code/distance/mmd_numpy_sklearn.py

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/Transfer learning project/UCR_TS_Archive_2015/')

In [5]:
dataset_list = ['50words/50words_TRAIN', 'Meat/Meat_TRAIN', 'HandOutlines/HandOutlines_TRAIN', 
                'InlineSkate/InlineSkate_TRAIN', 'DiatomSizeReduction/DiatomSizeReduction_TRAIN']

data = []
for data_name in dataset_list:
  data.append(read_csv(data_name, header=None))

In [6]:
np.shape(data[1])

(60, 449)

In [ ]:
"""
The algorithm is as follows:

Mix the two datasets. Apply label that indicate each example's origin.
Train a classifier on these merged data.
Measure the classifier's error e on a held-out test set.
Set PAD = 2 (1 − 2e)
"""

"\nThe algorithm is as follows:\n\nMix the two datasets. Apply label that indicate each example's origin.\nTrain a classifier on these merged data.\nMeasure the classifier's error e on a held-out test set.\nSet PAD = 2 (1 − 2e)\n"

In [8]:
def proxy_a_distance(source_X, target_X, verbose=False):
    """
    Compute the Proxy-A-Distance of a source/target representation
    """
    nb_source = np.shape(source_X)[0]
    nb_target = np.shape(target_X)[0]

    if verbose:
        print('PAD on', (nb_source, nb_target), 'examples')

    # C_list = np.logspace(-7, 5, 10)
    C_list = [0.0001, 0.000001]

    if np.shape(source_X)[1] < np.shape(target_X)[1]:
      half_source, half_target = int(nb_source/2), int(nb_target/2)
      train_X = np.vstack((source_X.iloc[0:half_source, :np.shape(source_X)[1]], target_X.iloc[0:half_target, :np.shape(source_X)[1]]))
      train_Y = np.hstack((np.zeros(half_source, dtype=int), np.ones(half_target, dtype=int)))

      test_X = np.vstack((source_X.iloc[half_source:, :np.shape(source_X)[1]], target_X.iloc[half_target:, :np.shape(source_X)[1]]))
      test_Y = np.hstack((np.zeros(nb_source - half_source, dtype=int), np.ones(nb_target - half_target, dtype=int)))

    else:
      half_source, half_target = int(nb_source/2), int(nb_target/2)
      train_X = np.vstack((source_X.iloc[0:half_source, :np.shape(target_X)[1]], target_X.iloc[0:half_target, :np.shape(target_X)[1]]))
      train_Y = np.hstack((np.zeros(half_source, dtype=int), np.ones(half_target, dtype=int)))  

      test_X = np.vstack((source_X.iloc[half_source:, :np.shape(target_X)[1]], target_X.iloc[half_target:, :np.shape(target_X)[1]]))
      test_Y = np.hstack((np.zeros(nb_source - half_source, dtype=int), np.ones(nb_target - half_target, dtype=int)))


    best_risk = []
    for C in C_list:
        clf = svm.SVC(C=C, kernel='linear', verbose=False)
        clf.fit(train_X, train_Y)

        train_risk = np.mean(clf.predict(train_X) != train_Y)
        test_risk = np.mean(clf.predict(test_X) != test_Y)

        if verbose:
            print('[ PAD C = %f ] train risk: %f  test risk: %f' % (C, train_risk, test_risk))

        if test_risk > .5:
            test_risk = 1. - test_risk

        best_risk.append(2 * (1. - 2 * test_risk))

    return best_risk

In [10]:
count = 1
for dt in data:
  for dtt in data:
    if dt.shape[0] != dtt.shape[0]:
      print("Combo: {}".format(count))
      count += 1
      print("the PAD values are: {}".format(proxy_a_distance(dt, dtt, verbose=True)))

Combo: 1
PAD on (450, 60) examples
[ PAD C = 0.000100 ] train risk: 0.117647  test risk: 0.117647
[ PAD C = 0.000001 ] train risk: 0.117647  test risk: 0.117647
the PAD values are: [1.5294117647058822, 1.5294117647058822]
Combo: 2
PAD on (450, 370) examples
[ PAD C = 0.000100 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 0.000001 ] train risk: 0.451220  test risk: 0.451220
the PAD values are: [2.0, 0.19512195121951215]
Combo: 3
PAD on (450, 100) examples
[ PAD C = 0.000100 ] train risk: 0.181818  test risk: 0.181818
[ PAD C = 0.000001 ] train risk: 0.181818  test risk: 0.181818
the PAD values are: [1.2727272727272727, 1.2727272727272727]
Combo: 4
PAD on (450, 16) examples
[ PAD C = 0.000100 ] train risk: 0.034335  test risk: 0.034335
[ PAD C = 0.000001 ] train risk: 0.034335  test risk: 0.034335
the PAD values are: [1.8626609442060085, 1.8626609442060085]
Combo: 5
PAD on (60, 450) examples
[ PAD C = 0.000100 ] train risk: 0.117647  test risk: 0.117647
[ PAD C = 0.000001 ] train